# Modèle GPBOOST

## Définition des variables utilisés dans le modèle

Variables de Production
Departement : Le département du fichier d'origine des données de production.\
Crops_sub : Sous-catégorie de cultures du fichier d'origine des données de production.\
Crops : Une des 14 catégories de cultures.\
New_Region : Région du fichier d'origine des données de production.\
Old_Region : Les anciennes régions du fichier Météo.\
Year : Année de production.\
Production : Production totale de la culture pour l'année.\
Rendement : Rendement de production, calculé comme la production divisée par la surface.\
Surface : Surface de production de la sous-catégorie de cultures pour le département et l'année sélectionnés.\
region : Région du département.


Variables Météo : https://www.historique-meteo.net/

1/ temp_moyenne : Température moyenne par région par année.\
2/ temp_max : Température maximale par région par année.\
3 /temp_min : Température minimale par région par année.\
4/ vitesse_vent : Vitesse moyenne du vent par région par année.\
5/ temp_vent : Température moyenne du vent par région par année.\
6 /precip_sum : Somme des précipitations par région par année.\
7/ record_precip : Précipitations maximales enregistrées par région par année.\
8/ humidite_moy : Humidité moyenne par région par année.\
9/ visibilite_moy : Visibilité moyenne par région par année.\
10/ couv_nuageuse_moy : Couverture nuageuse moyenne par région par année.\
11/ jours_chaleur : Nombre de jours plus chauds que la moyenne annuelle par région.\
12/ jours_froid : Nombre de jours plus froids que la moyenne annuelle par région.\
13/ jours_humidite_basse : Nombre de jours plus humides que la moyenne annuelle par région.\
14 /jours_precip_consecutifs : Nombre de jours consécutifs de précipitations supérieures à la moyenne annuelle par région.

In [1]:
#!pip install gpboost

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gpboost as gpb

## Charger et Préparer les Données

In [12]:

# Charger les données
file_path = 'Crops_Data.csv'
data = pd.read_csv(file_path, delimiter=',', encoding='utf-8')

# Afficher les colonnes disponibles pour vérification
print("Colonnes disponibles : ", data.columns)

# Extraction des informations de département
if 'Departement' in data.columns:
    data['Departement_Code'] = data['Departement'].str.split(' - ').str[0]
    data['Departement_Name'] = data['Departement'].str.split(' - ').str[1]
else:
    print("La colonne 'Departement' est manquante")

# Assurez-vous que les codes des départements sont des chaînes de caractères
data['Departement_Code'] = data['Departement_Code'].astype(str).str.zfill(3)

# Ajouter les coordonnées des départements
departments_gps = {
    '001': {'latitude': 46.1333, 'longitude': 5.35},
    '002': {'latitude': 49.5, 'longitude': 3.5},
    '003': {'latitude': 46.3, 'longitude': 3.0833},
    '004': {'latitude': 44.0833, 'longitude': 6.2333},
    '005': {'latitude': 44.6667, 'longitude': 6.3333},
    '006': {'latitude': 43.9333, 'longitude': 7.1833},
    '007': {'latitude': 44.75, 'longitude': 4.5},
    '008': {'latitude': 49.75, 'longitude': 4.5833},
    '009': {'latitude': 42.9667, 'longitude': 1.6},
    '010': {'latitude': 48.3, 'longitude': 4.1667},
    '011': {'latitude': 43.0833, 'longitude': 2.4167},
    '012': {'latitude': 44.25, 'longitude': 2.55},
    '013': {'latitude': 43.5, 'longitude': 5.4167},
    '014': {'latitude': 49.1333, 'longitude': -0.25},
    '015': {'latitude': 45.0333, 'longitude': 2.55},
    '016': {'latitude': 45.6667, 'longitude': 0.3333},
    '017': {'latitude': 45.75, 'longitude': -0.75},
    '018': {'latitude': 47.0667, 'longitude': 2.4167},
    '019': {'latitude': 45.3667, 'longitude': 1.9167},
    '02A': {'latitude': 41.6667, 'longitude': 8.9167},
    '02B': {'latitude': 42.45, 'longitude': 9.0833},
    '021': {'latitude': 47.35, 'longitude': 4.8333},
    '022': {'latitude': 48.5, 'longitude': -2.75},
    '023': {'latitude': 46.1167, 'longitude': 2.0333},
    '024': {'latitude': 45.1833, 'longitude': 0.7167},
    '025': {'latitude': 47.25, 'longitude': 6.3333},
    '026': {'latitude': 44.6667, 'longitude': 4.8333},
    '027': {'latitude': 49.0167, 'longitude': 1.15},
    '028': {'latitude': 48.3333, 'longitude': 1.3333},
    '029': {'latitude': 48.3333, 'longitude': -4.0833},
    '030': {'latitude': 44.0667, 'longitude': 4.3333},
    '031': {'latitude': 43.6667, 'longitude': 1.4333},
    '032': {'latitude': 43.6667, 'longitude': 0.5833},
    '033': {'latitude': 44.8333, 'longitude': -0.5833},
    '034': {'latitude': 43.6667, 'longitude': 3.5},
    '035': {'latitude': 48.0833, 'longitude': -1.6833},
    '036': {'latitude': 46.8833, 'longitude': 1.7},
    '037': {'latitude': 47.3333, 'longitude': 0.6667},
    '038': {'latitude': 45.25, 'longitude': 5.3333},
    '039': {'latitude': 46.75, 'longitude': 5.75},
    '040': {'latitude': 44.0, 'longitude': -1.25},
    '041': {'latitude': 47.5, 'longitude': 1.3333},
    '042': {'latitude': 45.5, 'longitude': 4.25},
    '043': {'latitude': 45.0, 'longitude': 3.5},
    '044': {'latitude': 47.3333, 'longitude': -1.6667},
    '045': {'latitude': 47.9167, 'longitude': 2.0833},
    '046': {'latitude': 44.6667, 'longitude': 1.4167},
    '047': {'latitude': 44.3833, 'longitude': 0.5},
    '048': {'latitude': 44.6667, 'longitude': 3.5},
    '049': {'latitude': 47.5, 'longitude': -0.5},
    '050': {'latitude': 49.0, 'longitude': -1.3333},
    '051': {'latitude': 49.0833, 'longitude': 4.0333},
    '052': {'latitude': 48.0, 'longitude': 5.3333},
    '053': {'latitude': 48.25, 'longitude': -0.6667},
    '054': {'latitude': 48.6667, 'longitude': 6.1667},
    '055': {'latitude': 49.0, 'longitude': 5.3833},
    '056': {'latitude': 47.75, 'longitude': -2.75},
    '057': {'latitude': 49.0, 'longitude': 6.1667},
    '058': {'latitude': 47.0, 'longitude': 3.5},
    '059': {'latitude': 50.5, 'longitude': 3.0},
    '060': {'latitude': 49.3333, 'longitude': 2.5833},
    '061': {'latitude': 48.5, 'longitude': 0.1667},
    '062': {'latitude': 50.5, 'longitude': 2.5},
    '063': {'latitude': 45.75, 'longitude': 3.0833},
    '064': {'latitude': 43.3333, 'longitude': -0.75},
    '065': {'latitude': 43.0833, 'longitude': 0.0833},
    '066': {'latitude': 42.6667, 'longitude': 2.75},
    '067': {'latitude': 48.6667, 'longitude': 7.6667},
    '068': {'latitude': 47.8333, 'longitude': 7.25},
    '069': {'latitude': 45.75, 'longitude': 4.6667},
    '070': {'latitude': 47.75, 'longitude': 6.1667},
    '071': {'latitude': 46.6667, 'longitude': 4.4167},
    '072': {'latitude': 48.0, 'longitude': 0.1667},
    '073': {'latitude': 45.5, 'longitude': 6.3333},
    '074': {'latitude': 46.0, 'longitude': 6.3333},
    '076': {'latitude': 49.922992, 'longitude': 1.077483},
    '077': {'latitude': 48.6667, 'longitude': 2.8333},
    '078': {'latitude': 48.8333, 'longitude': 1.9667},
    '079': {'latitude': 46.6667, 'longitude': -0.3333},
    '080': {'latitude': 50.0, 'longitude': 2.3333},
    '081': {'latitude': 43.75, 'longitude': 2.1667},
    '082': {'latitude': 44.0, 'longitude': 1.3333},
    '083': {'latitude': 43.4667, 'longitude': 6.3333},
    '084': {'latitude': 44.0, 'longitude': 5.0},
    '085': {'latitude': 46.5, 'longitude': -1.6667},
    '086': {'latitude': 46.5, 'longitude': 0.3333},
    '087': {'latitude': 45.8333, 'longitude': 1.3333},
    '088': {'latitude': 48.0833, 'longitude': 6.4167},
    '089': {'latitude': 47.8333, 'longitude': 3.5},
    '090': {'latitude': 47.75, 'longitude': 6.8333},
    '091': {'latitude': 48.6667, 'longitude': 2.25},
    '092': {'latitude': 48.8333, 'longitude': 2.25},
    '093': {'latitude': 48.9167, 'longitude': 2.4167},
    '094': {'latitude': 48.7833, 'longitude': 2.4667},
    '095': {'latitude': 49.05, 'longitude': 2.2167}
}

# Supprimer les départements d'outre-mer
outre_mer_departments = ['971', '972', '973', '974', '976']
data = data[~data['Departement_Code'].isin(outre_mer_departments)]

# Supprimer les lignes avec des coordonnées GPS manquantes
data = data.dropna(subset=['Longitude', 'Latitude'])

# Ajouter les coordonnées des départements au DataFrame
data['Latitude'] = data['Departement_Code'].map(lambda x: departments_gps.get(x, {}).get('latitude', 0))
data['Longitude'] = data['Departement_Code'].map(lambda x: departments_gps.get(x, {}).get('longitude', 0))

# Afficher les premières lignes pour vérifier
print(data.head())

# Sauvegarder le DataFrame nettoyé et mis à jour dans un nouveau fichier CSV
data.to_csv('Crops_Data_Cleaned.csv', index=False)



Colonnes disponibles :  Index(['Departement', 'Crops_sub', 'Crops', 'New_Region', 'Old_Region', 'Year',
       'Production', 'Rendement', 'Surface', 'region', 'year', 'temp_moyenne',
       'temp_max', 'temp_min', 'vitesse_vent', 'temp_vent', 'precip_sum',
       'record_precip', 'humidite_moy', 'visibilite_moy', 'couv_nuageuse_moy',
       'jours_chaleur', 'jours_froid', 'jours_humidite_basse',
       'jours_precip_consecutifs'],
      dtype='object')


KeyError: ['Longitude', 'Latitude']

## Préparer les Données pour l'Entraînement

In [11]:
# Préparer les données pour le modèle GPBoost
features = [
    'Longitude', 'Latitude', 'Year', 'temp_moyenne', 'temp_max', 'temp_min',
    'vitesse_vent', 'temp_vent', 'precip_sum', 'record_precip', 'humidite_moy',
    'visibilite_moy', 'couv_nuageuse_moy', 'jours_chaleur', 'jours_froid',
    'jours_humidite_basse', 'jours_precip_consecutifs'
]

# Séparer les données en ensembles d'entraînement et de test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

X_train = train_data[features]
y_train = train_data['Rendement']
X_test = test_data[features]
y_test = test_data['Rendement']

# Créer l'ensemble de données pour GPBoost
data_train = gpb.Dataset(X_train, label=y_train)

# Définir les paramètres du modèle GPBoost
params = {
    'objective': 'regression',
    'learning_rate': 0.1,
    'max_depth': 6,
    'num_leaves': 31,
    'verbose': 0
}

# Vérifier la taille des données pour éviter les problèmes de mémoire
print("Taille des données d'entraînement :", X_train.shape)

# Entraîner le modèle GPBoost avec dépendances spatiales
gp_model = gpb.GPModel(gp_coords=train_data[['Longitude', 'Latitude']].values, cov_function="exponential")
bst = gpb.train(params=params, train_set=data_train, gp_model=gp_model, num_boost_round=100)


# Vérifier les résultats
print("Modèle entraîné avec succès")


GPBoostError: std::bad_alloc

In [ ]:
# Prédictions
y_pred_train = bst.predict(data=X_train, gp_coords_pred=train_data[['Longitude', 'Latitude']].values)
y_pred_test = bst.predict(data=X_test, gp_coords_pred=test_data[['Longitude', 'Latitude']].values)

# Évaluation du modèle
from sklearn.metrics import mean_squared_error, r2_score

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"MSE train: {mse_train:.4f}, R2 train: {r2_train:.4f}")
print(f"MSE test: {mse_test:.4f}, R2 test: {r2_test:.4f}")

# Afficher quelques prédictions
print("Quelques prédictions sur l'ensemble de test :")
print(y_pred_test[:10])

In [3]:
from sklearn.model_selection import train_test_split
import gpboost as gpb

# Séparer les données en ensembles d'entraînement et de test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

X_train = train_data[['Surface', 'Year']]
y_train = train_data['Production']
X_test = test_data[['Surface', 'Year']]
y_test = test_data['Production']
group_train = train_data['Departement_Code'].values
group_test = test_data['Departement_Code'].values

# Créer l'ensemble de données pour GPBoost
data_train = gpb.Dataset(X_train, label=y_train)


## Définir et Entraîner le Modèle GPBoost

In [5]:
import pandas as pd
import numpy as np
import gpboost as gpb
from sklearn.model_selection import train_test_split

# Séparer les données en ensembles d'entraînement et de test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Utiliser un sous-ensemble des données pour tester
subset_train_data = train_data.sample(frac=0.1, random_state=42)  # Utiliser 10% des données pour l'entraînement
X_train_subset = subset_train_data[['Surface', 'Year']]
y_train_subset = subset_train_data['Production']
group_train_subset = subset_train_data['Departement_Code'].values

data_train_subset = gpb.Dataset(X_train_subset, label=y_train_subset)

# Définir le modèle de processus gaussien pour les effets de groupe et spatiaux
gp_model = gpb.GPModel(group_data=group_train_subset, gp_coords=subset_train_data[['Longitude', 'Latitude']].values)

# Spécifier les paramètres du modèle de boosting avec des ajustements
params = {
    'objective': 'regression_l2',
    'learning_rate': 0.1,
    'max_depth': 6,
    'num_leaves': 64,  # Ajuster le nombre de feuilles
    'min_data_in_leaf': 5,
    'verbose': 1
}

# Entraîner le modèle avec les nouveaux paramètres
num_boost_round = 100
bst = gpb.train(params=params, train_set=data_train_subset, gp_model=gp_model, num_boost_round=num_boost_round)

# Vérifier les résultats
print("Modèle entraîné avec succès")

[GPBoost] [Info] Total Bins 270
[GPBoost] [Info] Number of data points in the train set: 3235, number of used features: 2
[GPBoost] [Info] [GPBoost with gaussian likelihood]: initscore=345312.646677
[GPBoost] [Info] Start training from score 345312.646677
Modèle entraîné avec succès


## Faire des Prédictions

In [8]:
# Définir les ensembles de test
X_test = test_data[['Surface', 'Year']]
y_test = test_data['Production']
group_test = test_data['Departement_Code'].values
coords_test = test_data[['Longitude', 'Latitude']].values

# Faire des prédictions sur les données de test
pred = bst.predict(data=X_test, group_data_pred=group_test, gp_coords_pred=coords_test)
y_pred = pred['response_mean']


## Évaluer les Performances du Modèle

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculer l'erreur quadratique moyenne (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

# Calculer l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# Calculer le coefficient de détermination (R^2)
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

Root Mean Squared Error (RMSE): 1217141.535752782
Mean Absolute Error (MAE): 527429.7240467238
R^2 Score: 0.04920800799792402
